# Heston Volatility Approximation

In this notebook we implement the Heston implied volatility approximation via multivariate Chebyshev interpolation.

The analysis is structured as follows:

1. Compare results for the multivariate Chebyshev implementations using Numpy, Tensorflow and Julia.

2. Analyse approximation accuracy for Heston implied volatility use case.

## Summary of Heston Model

We summarise the Heston model to put our example and parametrisation choices into the wider context of implied volatility modelling. Heston model is a model for the price process of a financial asset $S_t$. The price process $S_t$ is described by the diffusion
$$
\begin{aligned}
  d S_t   &= \mu S_t dt + \sqrt{v_t} S_t d W^S_t, \\
  d v_t &= \kappa \left(\theta - v_t\right) dt + \xi \sqrt{v_t} d W^v_t, \\
  d W^S_t d W^v_t &= \rho dt
\end{aligned}
$$
with initial conditions $S_0>0$ and $v_0>0$ at $t=0$ and correlated Brownian motions $W^S_t$ and $W^v_t$.

For option pricing methods in Heston model we refer to the standard literature. Implied volatilities for a given term (or time to maturity) $T$ and strike price $K$ are obtained by inverting Black's formula given a forward Vanilla option price derived in Heston model.

For implied volatility modelling we are interested in forward prices. As a consequence, we can disregard the drift $\mu$. Implied volatility in Heston model is driven by the parameters of the squared volatility process $v_t$. In particular, we have the following properties.

  - $\sqrt{v_0}$ controls short term volatility.
  - $\sqrt{\theta}$ controls long term volatility.
  - $\log(2) / \kappa$ represents the half life of the expectation of $v_t$ moving from $v_0$ to $\theta$.
    This controls the term structure of at-the-money volatilities.
  - $\rho$ controls the volatility skew (or volatility slope in strike direction).
  - $\xi$ controls the volatility smile (or volatility curvature in strike direction).

For our parametrisation of implied volatilities we will use two properties of Heston model.

**Expectation of squared volatility.**

The expectation of the squared volatility $v_T$ for a given term $T\geq 0$ is given as
$$
  \mathbb{E} \left[ v_T \right] = v_0 e^{-\kappa T} + \theta \left(1 - e^{-\kappa T}\right).
$$
We use this this property to define an *average standard deviation* of asset prices as
$$
  \nu_{(v_0, \theta, \kappa)}(T) = \sqrt{\left[v_0 e^{-\kappa T} + \theta \left(1 - e^{-\kappa T}\right)\right] T}.
$$
The average standard deviation is used to normalise option strikes $K$. That is, We define the option moneyness as
$$
  {\cal M} = \frac{\log\left(K/S_0\right)}{\nu_{(v_0, \theta, \kappa)}(T)}.
$$

**Feller condition.**

The Feller condition for the squared volatility process is
$$
  \xi^2 \leq 2 \kappa \theta.
$$
This condition ensures that the squared volatility process remains positive, i.e. $v_t > 0$ for $t>0$. Violation of Feller condition e.g. by high vol-of-vol parameter $\xi$ is typically accepted to achieve reasonable fits in calibrations. However, high vol-of-vol parameters may cause numerical instabilities.

In order to control (or limit) the extend of Feller condition violation we use a *Feller factor* parameter to parametrise volatility smile. We define the Feller factor as
$$
  {\cal F} = \frac{\xi^2}{2 \kappa \theta}.
$$ 

**Implied volatility function parametrisation**

For a given model or market observation implied volatility $\sigma_{IV}$ is a function of the option term $T$ and the strike price $K$. That is
$$
  \sigma_{IV}\left(T, K\right)
$$
also forms a volatility surface.

For this analysis we extend the volatility surface function by the Heston model parameters and apply a parameter transformation. Our target function is $f:{\cal D}\rightarrow R$ with ${\cal D} \subset \mathbb{R}^8$ such that
$$
  f\left(x\right) = \sigma_{IV}\left(T, K; S_0, v_0, \theta, \kappa, \rho, \xi \right).
$$
The target function argument $x$ is specified as
$$
  x = \left[
  \begin{array}{c}
    x_0 \\ x_1 \\ x_2 \\ x_3 \\ x_4 \\ x_5 \\ x_6 \\ x_7
  \end{array}
  \right]
  = \left[
  \begin{array}{c}
    T \\ {\cal M} \\ S_0 \\ \sqrt{v_0} \\ \sqrt{\theta / v_0} \\ 0.7 / \kappa \\ \rho \\ {\cal F}
  \end{array}
  \right].
$$
The input domain ${\cal D}$ is the hyper-rectangle with boundaries as specified below:

In [ ]:
label_dict = {
    'term'            : ( 1/12, 5.0  ),
    'moneyness'       : ( -4.0, 4.0  ),
    'fwdPrice'        : ( 0.50, 1.50 ),
    'initial_vol'     : ( 0.10, 0.50 ),
    'long_vol_ratio'  : ( 0.50, 2.00 ),
    'decay_half_life' : ( 1.00, 5.00 ),
    'rho'             : (-0.80, 0.80 ),
    'feller_factor'   : ( 0.01, 4.00 ),
}

Some representative volatility smiles in Heston model are illustrated in the graph below.

In [ ]:
from ipywidgets import interact
from heston_volatility_plots import plot_smiles
from heston_volatility_plots import get_widgets
interact(
    plot_smiles,
    **get_widgets([1/12, 0.5, 1.0, 2.0], label_dict)
    )